In [1]:
import pandas as pd
import numpy as np
import os
import time, datetime
import matplotlib.pyplot as plt

In [2]:
#2020-11-02 18:51:24.674845
#67884674845
def get_time_stamp(time):
    hh = int(time[11:11+2])
    mm = int(time[14:14+2])
    ii = int(time[17:17+2])
    res = hh*3600+mm*60+ii
    for i in range(20,len(time)):
        res=res*10+ord(time[i])-ord('0')
    return res

In [3]:
train_data_c = pd.read_csv('CCF2021_run_record_c_Train.csv')
train_data_p = pd.read_csv('CCF2021_run_record_p_Train.csv')
train_data_label = pd.read_csv('CCF2021_run_label_Train.csv')
test_data_c = pd.read_csv('CCF2021_run_record_c_EvalA.csv')
test_data_p = pd.read_csv('CCF2021_run_record_p_EvalA.csv')

In [4]:
label_p_id = train_data_label['人员编号']
label_c_id = train_data_label['特征码']
answer = {label_p_id[i]:label_c_id[i] for i in range(len(label_p_id))}

In [5]:
%%time
train_data_c.sort_values(by = '发现时间',axis = 0,ascending = True,inplace=True)
train_data_p.sort_values(by = '发现时间',axis = 0,ascending = True,inplace=True)
test_data_c.sort_values(by = '发现时间',axis = 0,ascending = True,inplace=True)
test_data_p.sort_values(by = '发现时间',axis = 0,ascending = True,inplace=True)

Wall time: 284 ms


In [6]:
%%time
train_data_c['time'] = train_data_c['发现时间'].apply(lambda x:get_time_stamp(x))
train_data_p['time'] = train_data_p['发现时间'].apply(lambda x:get_time_stamp(x))
test_data_c['time'] = test_data_c['发现时间'].apply(lambda x:get_time_stamp(x))
test_data_p['time'] = test_data_p['发现时间'].apply(lambda x:get_time_stamp(x))

Wall time: 1.15 s


In [7]:
#train_data_c.drop(['经度','纬度'],axis=1,inplace=True)
#train_data_p.drop(['经度','纬度'],axis=1,inplace=True)

In [8]:
%%time
c_map = {}
p_map = {}
c_ids = []
p_ids = []
cid_len = {}
def init(data_c,data_p):
    global c_ids,p_ids,c_map,p_map,cid_len
    c_map = {}
    p_map = {}
    c_ids = []
    p_ids = []
    cid_len = {}
    c_ids = data_c['特征码'].unique().tolist()
    p_ids = data_p['人员编号'].unique().tolist()
    dfs = data_c.groupby(['特征码','点位编号'])
    for df in dfs:
        c_map[df[0]] = df[1].values.tolist()
    dfs = data_c.groupby(['特征码'])
    for df in dfs:
        c_map[df[0]] = df[1].values.tolist()
    dfs = data_c.groupby(['点位编号'])
    for df in dfs:
        c_map[df[0]] = df[1].values.tolist()
    dfs = data_p.groupby(['人员编号'])
    for df in dfs:
        p_map[df[0]] = df[1].values.tolist()
    for c_id in c_ids:
        cid_len[c_id] = len(c_map[c_id])
    print(len(c_ids),len(p_ids))

Wall time: 0 ns


In [9]:
###粗排###

In [10]:
cid_value_number = {}
def add_values(time,d_id,iters=1000):
    global c_ids,p_ids,c_map,p_map,cid_len,cid_value_number
    n = len(c_map[d_id])
    data_d_list = c_map[d_id]
    lb = 0
    ub = n-1
    ans=-1
    while ub>=lb:
        mid = (lb+ub) // 2
        if data_d_list[mid][5]<=time:
            ans=mid
            lb=mid+1
        else:
            ub=mid-1
    cid_set = set()
    for i in range(ans,-1,-1):
        cid = data_d_list[i][4]
        if cid in cid_set:
            continue
        cid_set.add(cid)
        value = time - data_d_list[i][5]
        if value > 80000000:
            break
        cid_value_number.setdefault(cid,[0,0])
        cid_value_number[cid][0]+=value
        cid_value_number[cid][1]+=1

In [11]:
def work(data_c,data_p,name):
    global c_ids,p_ids,c_map,p_map,cid_len,answer,cid_value_number
    init(data_c,data_p)
    new_c_ids = []
    new_p_ids = []
    answer_number,possible_number = 0,0
    label = []
    for p_id in p_ids:
        cid_value_number = {}
        data_p_list = p_map[p_id]
        for i in range(len(data_p_list)):
            add_values(data_p_list[i][5],data_p_list[i][0])
        f = 0
        pp_c_ids = []
        for c_id in cid_value_number:
            value = cid_value_number[c_id]
            pp_c_ids.append((c_id,(-value[1],cid_len[c_id],value[0])))
        pp_c_ids.sort(key=lambda x:x[1])
        for i in range(min(200,len(pp_c_ids))):
            c_id = pp_c_ids[i][0]
            possible_number+=1
            if c_id==answer.get(p_id,-1):
                answer_number+=1
                label.append(1)
            else:
                label.append(0)
            new_c_ids.append(c_id)
            new_p_ids.append(p_id)
    print(name,answer_number,possible_number)
    if name=='train':
        feature = pd.DataFrame({'c_id':new_c_ids,
                                'p_id':new_p_ids,
                                'label':label})
    else:
        feature = pd.DataFrame({'c_id':new_c_ids,
                                'p_id':new_p_ids})
    feature.to_csv(name+'_presort.csv',index=False)

In [12]:
%%time
work(train_data_c,train_data_p,'train')
work(test_data_c,test_data_p,'test')

10000 1997
train 1987 398338
10000 996
test 0 196382
Wall time: 22.8 s


In [13]:
df = pd.read_csv('train_presort.csv')
df.head(30)

,c_id,p_id,label
0,CY1r8Eov,P0099,0
1,CjD8XlTz,P0099,0
2,C1b6QpCZ,P0099,0
3,C1uhR4YE,P0099,0
4,CwhkYNEx,P0099,0
5,ChJfiEOd,P0099,0
6,C7GTsSLv,P0099,0
7,C3ukdWrT,P0099,0
8,CVUKwvDa,P0099,0
9,Cs3NVkxt,P0099,0
